# Training script

### Imports

In [1]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.multioutput import MultiOutputClassifier
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
from torch import nn
from tqdm import tqdm
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
import string
import fasttext
from sklearn.svm import SVC
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, pad_sequence
from sklearn.metrics import accuracy_score, classification_report

import os
os.makedirs('saved_models', exist_ok=True)

In [2]:
# model = MLP(37).cuda() # Replace model instantiation with another class here (SVC for example) if wishing to test other models
# model = XGBClassifier(n_estimators = 300, max_depth = 13, learning_rate = 0.01)
# model = XGBClassifier(n_estimators = 100, max_depth = 39 * 2, learning_rate = 0.01)
# model = SVC(C = 1, kernel = 'rbf', gamma = 'scale')
# TODO: somehting is wrong since the dict-based model achieves 0.30 accuracy
# and it only encounters 30/3000 not previously seen examples in the test set
# so it should have a 0.99 accuracy
TRANSFORMER_MODEL_NAME = 'bert-base-uncased'
SAVE_MODEL_SUFFIX = '1'
save_model_name = TRANSFORMER_MODEL_NAME.split('/')[-1]
transformer_tokenizer = AutoTokenizer.from_pretrained(TRANSFORMER_MODEL_NAME)

nr_features = 768
batch_size = 16
learning_rate = 2e-5
epochs = 10
patience = 2
class_weight_beta = 0.9999

In [3]:
def process_intent_list(intent_list):
    intents = set()
    if len(intent_list) == 0:
        intents.add('other')
    for intent in intent_list:
        if intent.startswith('Restaurant'):
            intents.add(intent)
        elif intent.startswith('Hotel'):
            intents.add(intent)
        elif intent.startswith('Booking'):
            intents.add(intent)
        elif intent.startswith('general'):
            intents.add(intent)
        else:
            intents.add('other')
    # print(f'Original {intent_list}')
    # print(f'Modified {list(intents)}')
    return list(intents)

def process_service_list(service_list):
    services = set()
    if len(service_list) == 0:
        services.add('other')
    for service in service_list:
        if service == 'restaurant':
            services.add('restaurant')
        elif service == 'hotel':
            services.add('hotel')
        else:
            services.add('other')
        if len(services) == 3:
            break
    return list(services)

### Loading the dataset

In [4]:
def preprocess_split(dataset, split):
    df = dataset[split].to_pandas()
    new_df = pd.DataFrame(columns = df.columns)
    for i in range(len(df)):
        # Taken from notebook, to know which lines to skip
        row = df.loc[i]
        if not any(set(row.turns['frames'][turn_id]['service']).intersection(['hotel', 'restaurant']) for turn_id,utt in enumerate(row.turns['utterance'])):
            continue
        
        new_df.loc[len(new_df)] = row
        # new_df.loc[len(new_df) - 1]['services'] = process_service_list(new_df.loc[len(new_df) - 1]['services'])
        # for i, frame_service in [frame['service'] for frame in df.loc[i].turns['frames']]:
            # df.loc[i].turns['frames']
    return new_df

def extract_to_be_retrieved_info(dataset):
    user_act_types_list = []
    user_slots_per_act_type_list = []
    to_be_retrieved_list = []
    tokens_list = []
    agent_act_types_list = []
    
    nr = 0
    for i in tqdm(range(len(dataset))):
        turns = dataset.loc[i].turns
        for j, (utterance, speaker, dialogue_act, frames) in enumerate(zip(turns['utterance'], turns['speaker'], turns['dialogue_acts'], turns['frames'])):
            # if speaker != 1:
                # continue
            # Skip using dialogue act intents
            # print(dialogue_act['dialog_act']['act_type'])
            # if 'other' in process_intent_list(dialogue_act['dialog_act']['act_type']):
                # continue
            # Skip using frame services
            # if 'other' in process_service_list(frames['service']):
                # continue
            services = frames['service']
            if speaker == 0:
                user_utterance = utterance
                current_booking_service = [service for service in services if service in ["hotel", "restaurant"]]
                
            act_types = dialogue_act['dialog_act']['act_type']
            act_slots = dialogue_act['dialog_act']['act_slots']
            
            # if speaker == 1 and not any(act_type.startswith("Hotel") or act_type.startswith("Restaurant") or act_type.startswith("Booking") for act_type in act_types):
            #     user_act_types_list.pop()
            #     user_slots_per_act_type_list.pop()
            #     continue
            
            # print(act_types)
            # if speaker == 0:
            #     # if len([1 for act_type in dialogue_act['dialog_act']['act_type'] if act_type.startswith('general')]) == len(dialogue_act['dialog_act']['act_type']):
            #         # skip_bot = True
            #         # continue
                
            #     if 'other' in process_intent_list(dialogue_act['dialog_act']['act_type']):
            #         skip_bot = True
            #         continue
            #     if 'other' in process_intent_list(turns['dialogue_acts'][j + 1]['dialog_act']['act_type']):
            #         skip_bot = True
            #         continue
            #     skip_bot = False
            # else:
            #     if skip_bot:
            #         continue
            if speaker == 0:
                skip_bot = False
                if not any(da.startswith("Hotel") or da.startswith("Restaurant") or da.startswith("Booking") for da in act_types):
                    skip_bot = True
                    continue
            elif skip_bot:
                    continue
            
            # print(act_slots)
            # print(act_types)
            slots_per_act_type = []
            to_be_retrieved = set()
            for act_type, slots in zip(act_types, act_slots):
                slot_names = slots['slot_name']
                slot_values = slots['slot_value']
                
                domain = act_type.split('-')[0].lower()
                if domain == 'booking' and len(current_booking_service)==1:
                    domain = current_booking_service[0]
                
                # to_be_retrieved.add(domain + '-availability')
                
                # if 'hotel' in domain or 'restaurant' in domain:
                if speaker == 0: # When it's the user's turn
                    for slot_name, slot_value in zip(slot_names, slot_values):
                        if slot_name != 'none':
                            slots_per_act_type.append(act_type.lower() + '-' + slot_name + ':' + slot_value)
                elif domain in ["hotel", "restaurant", "booking", "general"]: # When it's the bot's turn
                    act_type_relevant_slots = [(slot_name, slot_value) for slot_name, slot_value in zip(slot_names, slot_values) if slot_value != '?' and slot_name != 'none']
                    to_be_retrieved.update(set([domain + '-' + slot_name + ':' + slot_value for slot_name, slot_value in act_type_relevant_slots]))
                    
                    if len(to_be_retrieved) != 0 and any((slot_name_value.split(":")[0]!=domain+"-none" for slot_name_value in to_be_retrieved)) and not "-No" in act_types:
                        to_be_retrieved.add(domain + '-availability:yes')
                    elif "-No" in act_types:
                        to_be_retrieved.add(domain + '-availability:no')
                
            if speaker == 0: # When it's the user's turn
                user_act_types_list.append(act_types)
                user_slots_per_act_type_list.append(slots_per_act_type)
                
                # nr += 1
                # print(nr)
                # print("Input:", slots_per_act_type)
            else: # When it's the bot's turn
                to_be_retrieved_list.append(list(to_be_retrieved))
                agent_act_types_list.append(act_types)
                
                print(act_types)
                
                user_slots_per_act_type = user_slots_per_act_type_list[-1]
                to_be_retrieved = to_be_retrieved_list[-1]
                
                input_text = user_utterance + ' | ' + ', '.join(user_slots_per_act_type) + ' | ' + ', '.join(to_be_retrieved)
                print(input_text)
                
                tokenized = transformer_tokenizer(input_text, padding = 'max_length')
                
                tokens_list.append(tokenized)
            
    return tokens_list, agent_act_types_list

In [5]:
dataset = load_dataset('multi_woz_v22')

try:
    train
    print("Dataset already loaded, moving on")
except:
    train = preprocess_split(dataset, 'train')
    test = preprocess_split(dataset, 'test')
    val = preprocess_split(dataset, 'validation')
    train_embeddings_list, train_agent_act_types_list = extract_to_be_retrieved_info(train)
    test_embeddings_list, test_agent_act_types_list = extract_to_be_retrieved_info(test)
    val_embeddings_list, val_agent_act_types_list = extract_to_be_retrieved_info(val)

No config specified, defaulting to: multi_woz_v22/v2.2_active_only
Found cached dataset multi_woz_v22 (/home/adrian/.cache/huggingface/datasets/multi_woz_v22/v2.2_active_only/2.2.0/6719c8b21478299411a0c6fdb7137c3ebab2e6425129af831687fb7851c69eb5)


  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 762/762 [00:00<00:00, 999.69it/s] 


In [6]:
mlb = MultiLabelBinarizer().fit(train_agent_act_types_list)
# input_mlb = MultiLabelBinarizer().fit(train_user_slots_per_act_type_list)
print(mlb.classes_)

train_input = train_embeddings_list
train_output = mlb.transform(train_agent_act_types_list)

test_input = test_embeddings_list
test_output = mlb.transform(test_agent_act_types_list)

val_input = val_embeddings_list
val_output = mlb.transform(val_agent_act_types_list)
    

['Booking-Book' 'Booking-Inform' 'Booking-NoBook' 'Booking-Request'
 'Hotel-Inform' 'Hotel-NoOffer' 'Hotel-Recommend' 'Hotel-Request'
 'Hotel-Select' 'Restaurant-Inform' 'Restaurant-NoOffer'
 'Restaurant-Recommend' 'Restaurant-Request' 'Restaurant-Select'
 'general-bye' 'general-greet' 'general-reqmore' 'general-thank'
 'general-welcome']


In [7]:
samples_per_class = [0] * len(mlb.classes_)
for act_types in train_agent_act_types_list:
    for act_type in act_types:
        for i, class_ in enumerate(mlb.classes_):
            if class_ in act_type:
                samples_per_class[i] += 1

print("Class counts:")
print([*zip(mlb.classes_, samples_per_class)])

samples_per_class = np.array(samples_per_class)

effective_num = 1.0 - np.power(class_weight_beta, samples_per_class)
class_weights = (1.0 - class_weight_beta) / effective_num
class_weights = class_weights / np.sum(class_weights) * len(mlb.classes_)
print("Class weights:")
print([*zip(mlb.classes_, class_weights)])
class_weights = torch.Tensor(class_weights).cuda()

Class counts:
[('Booking-Book', 4030), ('Booking-Inform', 4300), ('Booking-NoBook', 1017), ('Booking-Request', 1927), ('Hotel-Inform', 6340), ('Hotel-NoOffer', 756), ('Hotel-Recommend', 1222), ('Hotel-Request', 2522), ('Hotel-Select', 842), ('Restaurant-Inform', 6390), ('Restaurant-NoOffer', 1179), ('Restaurant-Recommend', 1190), ('Restaurant-Request', 2419), ('Restaurant-Select', 749), ('general-bye', 5332), ('general-greet', 917), ('general-reqmore', 5951), ('general-thank', 25), ('general-welcome', 2913)]
Class weights:
[('Booking-Book', 0.10965330691605493), ('Booking-Inform', 0.1040676475033919), ('Booking-NoBook', 0.37611802146182216), ('Booking-Request', 0.2075105220236632), ('Hotel-Inform', 0.07746153667167696), ('Hotel-NoOffer', 0.4995042744756227), ('Hotel-Recommend', 0.31618596049903), ('Hotel-Request', 0.16316199144520221), ('Hotel-Select', 0.45039297129442024), ('Restaurant-Inform', 0.07702751351307752), ('Restaurant-NoOffer', 0.3270279583101872), ('Restaurant-Recommend', 

In [8]:
def batchify_tokens_tags(tokens_list, encoded_agent_tags_list, batch_size):
    ids_batch = []
    mask_batch = []
    useful_pos_batch = []
    labels_batch = []
    
    if encoded_agent_tags_list is None:
        encoded_agent_tags_list = range(len(tokens_list))
    
    for tokens, agent_encoded_tags in zip(tokens_list, encoded_agent_tags_list):
        ids_batch.append(tokens.input_ids)
        mask_batch.append(tokens.attention_mask)
        labels_batch.append(agent_encoded_tags)
        
        if len(ids_batch) == batch_size:
            yield torch.Tensor(ids_batch).long().cuda(), torch.Tensor(mask_batch).cuda(), torch.Tensor(labels_batch).cuda()
            ids_batch.clear()
            mask_batch.clear()
            labels_batch.clear()
    
    yield torch.Tensor(ids_batch).long().cuda(), torch.Tensor(mask_batch).cuda(), torch.Tensor(labels_batch).cuda()
    return None

def compute_loss(transformer, tokens_list, encoded_agent_tags_list, batch_size, criterion):
    transformer.eval()
    losses = []
    with torch.no_grad():
        for ids_batch, mask_batch, labels_batch in batchify_tokens_tags(tokens_list, encoded_agent_tags_list, batch_size):
            out = transformer.forward(input_ids = ids_batch, attention_mask = mask_batch)
            
            # logits_useful, labels_useful = outputs_keep_useful_part(out.logits, labels_batch, useful_pos_batch)
            logits_useful = out.logits
            labels_useful = labels_batch
            
            loss = criterion(logits_useful, labels_useful)
            losses.append(loss.item())
    transformer.train()
    return np.mean(losses)

def predict(transformer, tokens, batch_size):
    transformer.eval()
    predictions = []
    with torch.no_grad():
        for ids_batch, mask_batch, _ in batchify_tokens_tags(tokens, None, batch_size):
            out = transformer.forward(input_ids = ids_batch, attention_mask = mask_batch)
            res = (out.logits > 0).cpu().detach().numpy()
            predictions.append(res)
    return np.concatenate(predictions)

In [9]:
transformer = AutoModelForSequenceClassification.from_pretrained(TRANSFORMER_MODEL_NAME, num_labels = len(mlb.classes_), ignore_mismatched_sizes = True, problem_type = "multi_label_classification").cuda()
transformer.train()

criterion = nn.BCEWithLogitsLoss(weight = class_weights)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [10]:
optim = torch.optim.Adam(transformer.parameters(), lr = learning_rate)
train_losses = []
val_losses = []
waited = 0

min_val_loss = np.inf

for epoch in range(epochs):
    epoch_train_loss = []
    
    for ids_batch, mask_batch, labels_batch in batchify_tokens_tags(train_input, train_output, batch_size):
        optim.zero_grad()
        out = transformer.forward(input_ids = ids_batch, attention_mask= mask_batch)
        
        loss = criterion(out.logits, labels_batch)
        loss.backward()
        optim.step()
        epoch_train_loss.append(loss.item())
    
    epoch_train_loss = np.mean(epoch_train_loss)
    train_losses.append(epoch_train_loss)
    epoch_val_loss = compute_loss(transformer, val_input, val_output, batch_size, criterion)
    
    print(f"Epoch {epoch + 1}: Train loss = {epoch_train_loss}, Val loss = {epoch_val_loss}")
    
    if epoch_val_loss < min_val_loss:
        min_val_loss = epoch_val_loss
        transformer.save_pretrained('saved_models/MOVE_AGENT_ACTS_' + save_model_name + '_' + SAVE_MODEL_SUFFIX)
        
    
    if len(val_losses) != 0 and val_losses[-1] <= epoch_val_loss:
        waited += 1
        if waited > patience:
                val_losses.append(epoch_val_loss)
                break
    else:
        waited = 0
    
    val_losses.append(epoch_val_loss)

plt.plot(train_losses)
plt.plot(val_losses)
plt.show()


/tmp/ipykernel_6832/1329880963.py:16: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  yield torch.Tensor(ids_batch).long().cuda(), torch.Tensor(mask_batch).cuda(), torch.Tensor(labels_batch).long().cuda()


OutOfMemoryError: CUDA out of memory. Tried to allocate 384.00 MiB (GPU 0; 3.81 GiB total capacity; 3.21 GiB already allocated; 378.62 MiB free; 3.27 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
transformer = AutoModelForSequenceClassification.from_pretrained('saved_models/MOVE_AGENT_ACTS_' +  save_model_name + '_' + SAVE_MODEL_SUFFIX, num_labels = len(mlb.classes_)).cuda()

predicted_output = predict(transformer, test_input, batch_size)

acc = accuracy_score(test_output, predicted_output)
report = classification_report(test_output, predicted_output, target_names = mlb.classes_, digits = 3)
print(report)
print(f'acc = {acc}')

                       precision    recall  f1-score   support

      booking-bookday      0.000     0.000     0.000       109
   booking-bookpeople      0.000     0.000     0.000        94
     booking-bookstay      0.000     0.000     0.000        53
     booking-booktime      0.000     0.000     0.000        76
         booking-name      0.000     0.000     0.000       109
          booking-ref      0.726     0.707     0.717       499
        hotel-address      0.884     0.365     0.517       104
           hotel-area      0.509     0.117     0.191       247
       hotel-internet      0.000     0.000     0.000       155
           hotel-name      0.400     0.023     0.044       434
        hotel-parking      0.000     0.000     0.000       154
          hotel-phone      0.906     0.682     0.779        85
       hotel-postcode      0.909     0.606     0.727        66
     hotel-pricerange      1.000     0.008     0.017       236
            hotel-ref      0.000     0.000     0.000  

/home/adrian/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/adrian/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/adrian/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
